In [1]:
import sys
sys.path.append("../")

from logic.sms.entities import SingleMachineScheduling, Job, Machine
from logic.sms.cbnb import CombinatorialBnB
from logic.sms.sptf import SPTFRuleScheduler
from logic.sms.completions_sum import WeightedCompletionsSum
from logic.sms.gurobi import GurobiSolver, UpdateValue, SMS_LP_minWeightedSum_timeIndexed

from copy import deepcopy
from time import time
import csv
from func_timeout import func_set_timeout, FunctionTimedOut
import matplotlib.pyplot as plt
import numpy as np
from timeit import timeit
from cProfile import Profile
from pstats import Stats
from pyprof2calltree import convert, visualize

TIMEOUT = 60 * 10    # 10 minutes

def profile(p, objective, solver):
    
    error = None
    elapsed = None
    solverName = type(solver).__name__

    p.objective = objective
    p.solver = solver
    try:
        timeit_timeout = func_set_timeout(TIMEOUT)(timeit)
        elapsed = timeit_timeout("p.solve", globals={"p": p}, number=1)
        if p.value == None or p.value == np.inf or p.value == -np.inf or np.isnan(p.value):
             error = p.value
    except (Exception, FunctionTimedOut) as e:
        error = e
    return elapsed, error, solverName

def profileNJobs(job_sequences, dumpName):
        with open(dumpName + ".csv", "w") as output:
            writer = csv.DictWriter(output, ["solver", "jobs", "elapsed", "error"], lineterminator='\n', delimiter=',')
            writer.writeheader()
            for jobs in job_sequences:
                elapsed, error, solverName = profile(SingleMachineScheduling(deepcopy(jobs), Machine(0)), WeightedCompletionsSum(), CombinatorialBnB(SPTFRuleScheduler))
                writer.writerow({"solver": solverName, "jobs": len(jobs), "elapsed": elapsed, "error": error})
                elapsed, error, solverName = profile(SMS_LP_minWeightedSum_timeIndexed(deepcopy(jobs)), UpdateValue(), GurobiSolver())
                writer.writerow({"solver": solverName, "jobs": len(jobs), "elapsed": elapsed, "error": error})

def generateJobs(n, p=None, r=None, w=None, rngMethod=None, bounds=(0,1), **kwargs):

    def generateValue(value, rngMethod, default, bounds):

        if value is None and rngMethod is None:
            return default
        elif value is None and rngMethod is not None:
            return rngMethod(**kwargs) * (bounds[1] - bounds[0]) + bounds[0]
        else:
            return value

    
    return [Job(id=j, processingTime=generateValue(p, rngMethod, 1, bounds), releaseTime=generateValue(r, rngMethod, 0, bounds), weight=generateValue(w, rngMethod, 1, bounds)) for j in range(n)]

@func_set_timeout(timeout=TIMEOUT)
def profileSolverCallStack(p, output):
    profileFileName = f"{output}.prof"
    profile = Profile()
    profile.runctx(cmd="p.solve()", globals=globals(), locals={"p":p})
    profile.print_stats()
    stats = profile.getstats()
    convert(stats, f"{output}.kgrind")


In [9]:
# BnB vs Gurobi maximum input dimension

N_JOBS = (5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000)
DUMPNAME = f'inputDimension_5-50000_10m'

In [5]:
# dummy test
N_JOBS = (5, 10)
DUMPNAME = f'test'

In [5]:
# BnB vs Gurobi execution times

N_JOBS = (i for i in range (1, 21) for j in range (1000))
DUMPNAME = f'inputDimension_1-20'

In [6]:
job_sequences = [generateJobs(n, p=1, r=0, w=1) for n in N_JOBS]

In [12]:
# Random release time and weights

from numpy.random import default_rng

rng = default_rng(13061998)
job_sequences = [generateJobs(n=20, p=10, rngMethod=rng.uniform, bounds=(0, 50)) for i in range(1000)]
DUMPNAME = "randomProcessReleaseTime_20"

In [13]:
profileNJobs(job_sequences, DUMPNAME)

In [2]:
from numpy.random import default_rng

rng = default_rng(462001)
n = 100
jobs = generateJobs(n, p=10, rngMethod=rng.uniform, bounds=(0, n))

In [3]:
# build problem using bnb scheduler
p = SingleMachineScheduling(jobs, Machine(0))
p.objective = WeightedCompletionsSum()
p.solver = CombinatorialBnB(SPTFRuleScheduler)

# call profiler solver call stack
profileSolverCallStack(p, "bnb_100_callstack")


         145020799 function calls (119539094 primitive calls) in 75.408 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   75.408   75.408 <string>:1(<module>)
        1    0.126    0.126   74.907   74.907 bnb.py:31(__solve_bnb)
        1    0.000    0.000   75.408   75.408 bnb.py:41(__call__)
       57    0.185    0.003   29.052    0.510 cbnb.py:10(branch)
     5644    0.038    0.000    0.485    0.000 cbnb.py:52(isDominated)
     5644    0.159    0.000   46.157    0.008 cbnb.py:65(isFathomed)
     5645    0.166    0.000    0.234    0.000 completions_sum.py:6(__call__)
22054997/11288   26.287    0.000   55.712    0.005 copy.py:128(deepcopy)
 18577393    1.933    0.000    1.933    0.000 copy.py:182(_deepcopy_atomic)
1140188/11388    3.689    0.000   54.549    0.005 copy.py:201(_deepcopy_list)
1151876/11288    5.952    0.000   55.422    0.005 copy.py:227(_deepcopy_dict)
  3466316    1.869    0.000  